In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.YOLO_utils as YOLO
import utils.VOC_utils as VOC
from utils.lmdb_utils import *
from utils.eval_utils import *
from utils.image_utils import *
from utils.json_utils import parse
from models import get_model


In [ ]:
conf = parse('../config/YOLOV3_VOC_base.json')
device = conf["device"]
is_train = False
epoch = 120
exp_dir = conf["exp_dir"]
if epoch == 0:
    weights = '/4TB/aaron/data_yolo/yolo-voc.weights'
    conf["model"]["pretrain"] = weights

data_split = 'test'
filename = os.path.join(exp_dir, 'predict_{}_{}_'.format(epoch, data_split))
year = '2007'
classes = VOC.CLASS_LIST
img_root = conf["dataset"][data_split]["root"]
with open(img_root, 'r') as file:
    lines = file.readlines()    

In [ ]:
YOLO.do_python_voc_eval(filename, VOC.DIR, classes, year, data_split)

# Detection

In [ ]:
model = get_model(conf["model"], device, epoch, is_train, exp_dir, False)

idx = 0
imgpath = lines[idx].rsplit()[0]
img = Image.open(imgpath).convert('RGB')
img = img.resize((416, 416))
img = np.asarray(img)
nms_thresh = 0.45
conf_thresh = 0.05
pred_labels = YOLO.detect(model, img, conf_thresh, nms_thresh)
pred_labels = np.asarray(pred_labels)
labpath = imgpath.replace('images', 'labels').replace('JPEGImages', 'labels').replace('.jpg', '.txt').replace('.png','.txt')
true_labels = np.loadtxt(labpath)

In [ ]:
print(imgpath)
fig, ax = plt.subplots(1,2, figsize=(15, 15))
ax[0].imshow(img)
ax[1].imshow(img)
boxes = pred_labels[:, :4]
ax[0].set_title('pred')
VOC.draw_bbox(ax[0], boxes, (img.shape[1], img.shape[0]), class_id = pred_labels[:, -1])
ax[1].set_title('true')

VOC.draw_bbox(ax[1], true_labels[:, -4:], (img.shape[1], img.shape[0]), class_id = true_labels[:, 0])